In [1]:
# Reference
# https://yurimkoo.github.io/python/2019/09/14/connect-db-with-python.html

import pandas as pd
import requests
import pymysql
import os
from dotenv import load_dotenv
from sqlalchemy import create_engine

os.chdir(os.path.dirname(os.getcwd()))

from functions.pubgapi import get_tournament_info
from functions.pubgapi import get_match_info
from functions.pubgapi import get_match_participant

os.chdir("./Example")

# Load api key & RDS info (endpoint="rds 엔드포인트", dbname="db 이름", username="마스터 사용자 이름", password="rds 비밀번호")
load_dotenv()
api_key = os.environ.get("API_KEY")
endpoint = os.environ.get("AWS_endpoint")
dbname = os.environ.get('AWS_dbname')
username = os.environ.get("AWS_username")
password = os.environ.get("AWS_password")

In [2]:
tournament_info = get_tournament_info(api_key)
print("tournament_info(dataframe):\n",tournament_info)

# DB와 connect
# create_engine("mysql+pymysql://아이디:"+"암호"+"@주소:포트/데이터베이스이름?charset=utf8", encoding='utf-8')
engine = create_engine("mysql+pymysql://admin:"+password+"@"+endpoint+":3306/"+dbname+"?charset=utf8", encoding="utf-8")
conn_create = engine.connect()

# AWS RDS에 tournament_info 테이블 create
tournament_info.to_sql(name='tournament_info', con=conn_create, if_exists='replace', index=False)
conn_create.close()

# DB connect
conn = pymysql.connect(host=endpoint, user=username, password=password, db=dbname, charset='utf8', port=3306)
cur = conn.cursor()

# 업로드된 테이블 확인(SQL 사용)
sql = "SHOW TABLES"
cur.execute(sql)
result = cur.fetchall()
print("\n\nSHOW TABLES: \n",result,"\n\n")

# 내용 확인(SELECT *)
sql = "SELECT * FROM tournament_info"
cur.execute(sql)
result1 = cur.fetchall()
result1 = pd.DataFrame(result1)
print("\ntournament_info(database):\n",result1)

tournament_info(dataframe):
              id             createdAt
0      eu-tgs22  2022-04-27T17:39:01Z
1      eu-trt22  2022-04-22T15:48:35Z
2    eu-goats22  2022-04-19T17:40:55Z
3      as-apls1  2022-04-17T10:16:50Z
4      eu-esapa  2022-04-16T16:38:55Z
..          ...                   ...
462  kr-pss18s2  2018-06-17T03:40:44Z
463  ct-mpl18s1  2018-06-15T09:29:17Z
464  kr-apl18s2  2018-06-15T07:21:54Z
465  jp-pjs18s1  2018-06-14T14:22:21Z
466  kr-pwm18s1  2018-06-13T15:38:29Z

[467 rows x 2 columns]


SHOW TABLES: 
 (('123',), ('49634508-f42a-4c39-a696-360347083736',), ('tournament_info',)) 



tournament_info(database):
               0                     1
0      eu-tgs22  2022-04-27T17:39:01Z
1      eu-trt22  2022-04-22T15:48:35Z
2    eu-goats22  2022-04-19T17:40:55Z
3      as-apls1  2022-04-17T10:16:50Z
4      eu-esapa  2022-04-16T16:38:55Z
..          ...                   ...
462  kr-pss18s2  2018-06-17T03:40:44Z
463  ct-mpl18s1  2018-06-15T09:29:17Z
464  kr-apl18s2  2018-06

In [3]:
# Save tournament_info.csv
# tournament_info.to_csv("./Data/tournament_info.csv")

conn.commit()

# db접속 종료
conn.close()